Called for detectors to export the bounding boxes in the testset. 

Additionally, it can at the same time apply a landmark localisation technique. 

In [ ]:
from __future__ import division
from os.path import isdir, join, isfile, sep
from os import listdir 
from functools import partial
from warnings import warn
import numpy as np
from research_pyutils.path_related_functions import mkdir_p, rm_if_exists
import logging
from pathlib import Path
# workerbee is applied in order to call condor.
from workerbee import exhaust_all_files_randomly

# menpo packages imports
import menpo.io as mio
from menpodetect import load_dlib_frontal_face_detector
from menpodetect import load_opencv_frontal_face_detector
from menpodetect.ffld2 import load_ffld2_frontal_face_detector
from menpodetect.dlib.conversion import pointgraph_to_rect
from dlib import shape_predictor
from menpo.shape import PointCloud
from menpo.landmark import LandmarkGroup

# Define paths and detection + landmark localisation method.

In [ ]:
path_base = '/vol/atlas/homes/grigoris/misc/2016_ijcv/data/300vw_testset/'
assert(isdir(path_base))

path_pickles = '/vol/atlas/homes/grigoris/misc/2016_ijcv/data/pickles/'
assert(isdir(path_pickles))

In [ ]:
method_detection = 'opencv'
# do not use with AAM, since it doesn't include the regression for the bounding boxes. 
method_landm_loc = 'sdm'

fold_out = 'detection_' + method_detection + '_' + method_landm_loc
save_bb = 1  # if set to 0, then it won't save the bb.
fold_det_out = 'detector_' + method_detection  # save bb from the method
detector = return_detector(method_detection)

In [ ]:
def return_detector(detection):
    """
    Returns a partial function with the detector based on the (string) detection
    that includes the name of the detector we wish to apply. 
    """
    if detection == 'dlib':
        return load_dlib_frontal_face_detector()
    elif detection == 'opencv':
        from functools import partial
        det = load_opencv_frontal_face_detector()
        return partial(det, min_neighbours=3)
    elif detection == 'ffld2':
        return load_ffld2_frontal_face_detector()
    else:
        raise RuntimeError('Not a valid choice of detection ({}).'.format(detection))

        
def detection_to_pointgraph(detection):
    return PointCloud(np.array([(p.y, p.x) for p in detection.parts()]))

In [ ]:
# load prediction model
if method_landm_loc == 'dlibERT':
    path_shape_pred = path_pickles + 'modelln_' +  method_detection + '_' + method_landm_loc + '.model'
    assert(isfile(path_shape_pred))
    model = shape_predictor(path_shape_pred)
else:
    path_pkl = path_pickles + 'modelln_' +  method_detection + '_' + method_landm_loc + '.pkl'
    assert(isfile(path_pkl))
    model = mio.import_pickle(path_pkl)

In [ ]:
def _get_path(p0, name):
    assert(isdir(p0))
    p1 = join(p0, name, '')
    if not isdir(p1):
        mkdir_p(p1)
    return p1
    
def process_frame(p_fr):
    # using global: detector(), p_ln_out_0, model, p_condor_dummy_0, p_bb_out_0, save_bb
    try:
        im = mio.import_image(p_fr)
        if im.n_channels == 3:
            im = im.as_greyscale()
        detector(im)
        assert(im.landmarks.n_groups < 10)  # might get complicated otherwise with the naming
        if im.landmarks.n_groups > 0:  # there are detections
            ln = im.landmarks[im.landmarks.group_labels[0]]
            
            if save_bb:
                p_bb_out = _get_path(p_bb_out_0, im.path.parent.name) + im.path.stem + '.pts'
                mio.export_landmark_file(ln, p_bb_out, overwrite=True)
            
            p_out = _get_path(p_ln_out_0, im.path.parent.name) + im.path.stem + '.pts'
            
            # ensure that the bb is inside the bounds
            im.landmarks['bb'] = ln
            im.constrain_landmarks_to_bounds()
            ln = im.landmarks['bb']

            if method_landm_loc == 'dlibERT':
                im_pili = np.array(im.as_PILImage())
                det_frame = model(im_pili, pointgraph_to_rect(ln.lms))
                init_pc = detection_to_pointgraph(det_frame)
                mio.export_landmark_file(LandmarkGroup.init_with_all_label(init_pc), 
                                         p_out, overwrite=True)
            else:
                ft = model.fit_from_bb(im, ln.lms)
                im.landmarks['gg'] = ft.final_shape
                mio.export_landmark_file(im.landmarks['gg'], p_out, overwrite=True)
        else:  # temp, can be removed, debugging purposes. 
            print("The input '{}' has no ln.".format(p_fr))
        
        # create dummy file for workerbee
        p_cond = _get_path(p_condor_dummy_0, im.path.parent.name)
        open(p_cond + im.path.stem + '.pts', 'a').close() 
    except Exception as e:
        print("The input '{}' failed.".format(p_fr))
        logging.exception("The input '{}' failed.".format(p_fr))
        raise ValueError()

# Execute for all categories, all frames.

In [ ]:
cats = listdir(path_base)
for cat in cats:
    if not cat[:8] == 'category' or not isdir(path_base + cat):
        warn('Unknown content in path {} (folder: {}).'.format(path_base, cat))
    print(cat)
    # join or create the paths
    p_cat = join(path_base, cat, '')
    p_fr = join(p_cat, 'frames', '')
    assert(isdir(p_fr))  # frames folder should exist
    p_ln_out_0 = mkdir_p(join(p_cat, fold_out, ''))
    p_bb_out_0 = mkdir_p(join(p_cat, fold_det_out, ''))
    # the condor path below is utilised just to save some dummy files. That 
    # way workerbee will know that this frame has been processed and can measure the progress.
    p_condor_dummy_0 = mkdir_p(join(p_cat, 'condor_tmp', 'condor_dummy_' + fold_out, ''))
    
    for c in sorted(listdir(p_fr)):   # for each clip
        output_dir = Path(mkdir_p(p_condor_dummy_0 + c + sep))
        done = lambda: output_dir.glob('*.pts')
        im_paths = lambda: mio.image_paths(p_fr + c + sep + '*')
        exhaust_all_files_randomly(im_paths, done, process_frame, verbose=True)
    # uncomment below, only if NOT called in condor.
    # rm_if_exists(p_condor_dummy_0)
del model